## Import package

In [1]:
import sys
%run  '/home/sun/ETGEMS-10.20/ETGEMs_function_protain.py'
import os
import pandas as pd
import cobra
import json
import copy
from cobra import Model, Reaction, Metabolite

## Setting path for models and data

In [ ]:
# data 
model_file="./data/lys.json" # c.glutamicum model, irreversible and reactions with isoezyme splited
reaction_kcat_MW_file='./data/reaction_change_by_enzuse_PDH_n.csv'
# results
path_results = './result'
dictionarymodel_path='./result/dictionarymodel_iCW.json' # dictionary model saving path
# case study
model0_file = '../data/iCW/iCW773_uniprot_modification_del.json'
reaction_g0_file=os.path.join('./task/iCW773_uniprot_modification_del_reaction_g0.csv')
metabolites_lnC_file = os.path.join('./task/metabolites_lnC_cg1.txt')

## load model and split model

In [3]:
model=cobra.io.load_json_model(model_file)
dictionary_model=trans_model2standard_json_etgem(model_file)
[reaction_list,metabolite_list,lb_list,ub_list,coef_matrix]=get_data(model)

./lys_irreversible.json


## get enzyme parameter

In [4]:
#get kcat_dict and mw_dict
kcat_mw=pd.read_csv(reaction_kcat_MW_file,index_col=0)
kcat_dict={}
mw_dict={}
for i in kcat_mw.index:
    if i in model.reactions:
        kcat_dict[i]=kcat_mw.loc[i,'kcat_1/h']
        enz=str(model.reactions.get_by_id(i).gpr)
        flag1=enz.split(' and ')
        flag1.sort(key=None, reverse=False)
        flag2=' and '.join(flag1)
        mw_dict[flag2]=kcat_mw.loc[i,'MW_g/mmol']

In [ ]:
kcat_mw

,kcat_1/h,MW_g/mmol,kcat_MW
ALAR,9.904612e+05,78.346447,12642.069812
ALATA_L,1.901828e+05,46.748856,4068.179773
ASNN,9.729920e+05,33.298224,29220.536266
ASNS1,7.336000e+03,72.387691,101.343197
ASPT,4.748529e+05,57.567954,8248.562351
...,...,...,...
PSERtex_reverse_num4,2.663573e+07,28.237185,943285.533462
URAt2rpp_reverse_num2,2.663573e+07,66.934412,397937.735374
PHETA1_reverse_num2,1.026000e+04,79.835554,128.514171
SHK3Dr_reverse_num2,4.380570e+05,29.304723,14948.341262


## setting kcat/mw


In [5]:
def add_parameter(dictionarymodel,kcatdata,kmdata,mwdata):
    for enz in dictionarymodel['enzyme']:
        if enz in mwdata.keys():
            dictionarymodel['enzyme'][enz]['MW']=mwdata[enz]
        for rea in dictionarymodel['enzyme'][enz]['reactions']:
            if  rea in kcatdata.keys():
                dictionarymodel['enzyme'][enz]['reactions'][rea]['kcat']=kcatdata[rea]
            if  rea in kmdata.keys():
                dictionarymodel['enzyme'][enz]['reactions'][rea]['km']=kmdata[rea]
    for enz in mwdata:
        if enz not in dictionarymodel['enzyme']:
            print(enz)

In [ ]:
add_parameter(dictionary_model,kcat_dict,{},mw_dict)
json_write(dictionarymodel_path,dictionary_model)